# Building a Custom Dense Layer

In this notebook, we'll walk through how to create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class. Unlike simple Lambda layers you did previously, the custom layer here will contain weights that can be updated during training.

## Imports

In [1]:
# try:
#     # %tensorflow_version only exists in Colab.
#     %tensorflow_version 2.x
# except Exception:
#     pass
from IPython.display import Image
from IPython.core.display import HTML 

import tensorflow as tf
import numpy as np

In [2]:
Image(url= "https://miro.medium.com/max/1826/1*L9xLcwKhuZ2cuS8fF0ZjwA.png")

## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [3]:
# inherit from this base class
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
#         print('Input shape', input_shape)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'), trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

Now we can use our custom layer like below:

In [20]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'simple_dense_12/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.00912529]], dtype=float32)>, <tf.Variable 'simple_dense_12/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Let's then try using it in simple network:

In [21]:
# y = 2*x - 1
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# # configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

[[18.981558]]
[<tf.Variable 'simple_dense_13/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.997327]], dtype=float32)>, <tf.Variable 'simple_dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([-0.99171275], dtype=float32)>]


In [7]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
x2 = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
y = []
for i in range(6):
    x = 2*xs[i] + 3*x2[i] -1
    y.append(x)
    
y

[-6.0, -1.0, 4.0, 9.0, 14.0, 19.0]

In [11]:
class SimpleDense_2(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        print('Input shape', input_shape)
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'), trainable=True)
        
#         self.w2 = tf.Variable(name="kernel2",
#             initial_value=w_init(shape=(input_shape[-1], self.units),
#                                  dtype='float32'), trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

In [22]:
# y = 2*x1 + 3*x2 - 1
# define the dataset
xs = np.array([[-1.0, -1.0],  [0.0, 0.0],  [1.0, 1.0],  [2.0, 2.0], [3.0, 3.0], [4.0, 4.0]], dtype=float)
ys = np.array([-6.0, -1.0, 4.0, 9.0, 14.0, 19.0], dtype=float)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# # configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

# perform inference
print(model.predict([[10.0, 10.0]]))
print(model.evaluate(xs, ys))

# see the updated state of the variables
print(my_layer.variables)

[[48.984974]]
1/1 [==============================] - 0s 73ms/step - loss: 3.0002e-05
3.0002067433088087e-05
[<tf.Variable 'simple_dense_14/kernel:0' shape=(2, 1) dtype=float32, numpy=
array([[2.4304926],
       [2.5672767]], dtype=float32)>, <tf.Variable 'simple_dense_14/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9927184], dtype=float32)>]
